# Recipe Recommender System

**Want a specific meal for within a certain budget for a specific number of people**

In [1]:
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

In [4]:
meal = 'breakfast'

In [5]:
print(meal.title())

Breakfast


In [ ]:
if meal == "1":
        meal = 'Breakfast'
    else:
        meal = 'Dinner'

    if dietary_preference == "1":
        dietary_preference = 'Vegetarian'
    else:
        dietary_preference = None

    for title in recipe_final:
        if meal in recipe_tags_dict[title]:
            restricted_recipes.append(title)

    for diet in restricted_recipes:
        if dietary_preference in recipe_tags_dict[diet]:
            ret.append(diet)

## Read in Files

In [2]:
data_matrix = pd.read_csv('../../data/05_model_output/data_matrix_sim.csv')

In [3]:
data_matrix.set_index('Unnamed: 0', inplace=True)

In [4]:
infile = open('../../data/03_processed/crf_ingred_dict','rb')
recipe_dict = pickle.load(infile)
infile.close()

In [5]:
infile = open('../../data/03_processed/crf_links_dict','rb')
recipe_links_dict = pickle.load(infile)
infile.close()

In [6]:
infile = open('../../data/03_processed/crf_tag_dict','rb')
recipe_tags_dict = pickle.load(infile)
infile.close()

## Price Groceries

In [7]:
grocery_prices_marianos = pd.read_csv('../../data/03_processed/grocery_prices_marianos_final.csv')

In [8]:
grocery_prices_marianos

,product,main_price,prod_aile,price_per_lb,measure_words_main_price,item_weight_count_vol,date_collected,store,location,product_new
0,Green Onions (Scallions),$0.99,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,scallions
1,Cilantro,$0.79,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,cilantro
2,Italian Parsley,$0.99,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,parsley
3,Roundy's Organic Fresh Rosemary,$2.50,Fresh Herbs,NaN,each,0.75 ounce,2019-08-28,Marianos,60615,roundys
4,Roundy's Organic Fresh Thyme,$2.50,Fresh Herbs,NaN,each,0.75 ounce,2019-08-28,Marianos,60615,thyme
5,Roundy's Mint,$2.50,Fresh Herbs,NaN,each,0.75 ounce,2019-08-28,Marianos,60615,mint
6,Basil,$1.79,Fresh Herbs,NaN,each,1 each,2019-08-28,Marianos,60615,basil
7,Dill,$1.99,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,dill
8,Roundy's Organics Fresh Dill,$2.50,Fresh Herbs,NaN,each,1 bunch,2019-08-28,Marianos,60615,dill
9,Gourmet Garden™ Ginger Stir-in Paste,$4.49,Fresh Herbs,NaN,each,4 ounce,2019-08-28,Marianos,60615,paste


## Recipe Recommendor System 

1. user builds a shopping cart
1. user specifies meal (breakfast, lunch, dinner, snack) 
1. <font color='red'>REACH GOAL</font>: user specifies dollar amount
1. from the shopping cart a large list of similar items are gathered
1. search the database for a recipe that has at least X of those ingredients with specified params

##### First get list of foods found in the database

In [9]:
# build a list of foods that are found in the food database
database_food_list = list(data_matrix.columns)

In [10]:
shopping_basket = ['cheese', 'chicken', 'pasta']

In [11]:
items_in_database = []
item_missing = []
for item in shopping_basket:
    if item in database_food_list:
        items_in_database.append(item)
    else:
        item_missing.append(item)
if len(item_missing) > 0:
    print('Items Not Found in Database: ', item_missing)

##### Search instacart database for similair foods 

In [12]:
known_user_likes = items_in_database
# Construct a new dataframe with the 10 closest neighbours (most similar)
# for each artist.
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,11))
for i in range(0, len(data_matrix.columns)):
    data_neighbours.ix[i,:10] = data_matrix.ix[0:,i].sort_values(ascending=False)[:10].index
# Get the artists the user has played.

# Construct the neighbourhood from the most similar items to the
# ones our user has already liked.
most_similar_to_likes = data_neighbours.loc[known_user_likes]
similar_list = most_similar_to_likes.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))

print(similar_list)
# need to remove bread and milk because they appear in everything 

['turkey', 'yogurt', 'cheese', 'banana', 'tomatoes', 'strawberries', 'milk', 'eggs', 'bread', 'avocado', 'apple', 'parmesan', 'chicken', 'pasta', 'pasta sauce']


In [13]:
most_similar_to_likes

,1,2,3,4,5,6,7,8,9,10
cheese,cheese,milk,bread,yogurt,chicken,banana,strawberries,eggs,avocado,apple
chicken,chicken,cheese,milk,banana,bread,avocado,yogurt,strawberries,turkey,eggs
pasta,pasta,cheese,pasta sauce,bread,chicken,milk,banana,tomatoes,eggs,parmesan


##### Search recipe with specific items 

In [11]:
from collections import Counter

def _extract_ingredients(single_recipe_ing_list):
    ret = []
    for ingredient in single_recipe_ing_list:
        ret.append(ingredient['name'])
    return ret

def _ingredient_recipe_matcher(ingredients_list, shopping_list, recipe_title):
    matching_recipes = []
    matching_ingredients = []
    return_recipes = []

    for element in shopping_list:
        if any(element in s for s in ingredients_list):
            matching_recipes.append(recipe_title)
        count_recipe_dict = dict(Counter(matching_recipes))

    for key, val in count_recipe_dict.items():
        if val >= 3:
            return_recipes.append(key)
    
    if len(return_recipes) > 0:
        return return_recipes

def find_matching_recipe(recipe_dict, shopping_list, meal = None, dietary_preference=None):
    listofkeys = []
    recipes = []
    recipe_final = []
    restricted_recipes = []
    ret = []
    listofitems = recipe_dict.items()

    for item in listofitems:
        ingred_list = _extract_ingredients(item[1])
        recipe_title = item[0]
        recipes.append(_ingredient_recipe_matcher(ingred_list, shopping_list, recipe_title))
    filtered_recipes = list(filter(None, recipes))
    
    for lst in filtered_recipes: 
        for item in lst:
            recipe_final.append(item)
    
    # check if meal and dietary preferences met
    if meal is not None:
        for title in recipe_final:
            if meal in recipe_tags_dict[title]:
                restricted_recipes.append(title)
    else:
        restricted_recipes = recipe_final
            
    if dietary_preference is not None:
        for diet in restricted_recipes:
            if dietary_preference in recipe_tags_dict[diet]:
                ret.append(diet)
    else:
        ret = restricted_recipes
        
    return ret

In [17]:
find_matching_recipe(recipe_dict, similar_list, meal='Dinner')

['italian grilled cheese sandwiches',
 'chicken panzanella salad',
 'chicken piccata pasta',
 'skillet chicken parmesan',
 'goldfish chicken tenders',
 'slow cooker shredded chicken chili',
 'cheesy bruschetta chicken cutlets',
 'baked chicken parmesan',
 'sesame chicken fingers with spicy orange dipping sauce',
 'stuffed shells with summer squash and ricotta',
 'mushroom toast with fried egg',
 'savory dutch baby pancake with salmon and fried egg',
 'chili mac and cheese',
 'fettuccine alfredo with mushrooms',
 'creamy baked mac and cheese',
 'vegetable shakshuka with pesto']

***

## Pricing Recipes

In [21]:
for key, value in recipe_dict.items():
    for item in value:
        print(item['qty'], item['unit'], item['name'])
#     for food in value['name']:
#         print(food)
# #         print('\ n')

8 slices sourdough bread
4 tablespoon unsalted butter
8 ounces 2 cups shredded cheddar cheese
2  slicing tomatoes
8 to 12 slices 
cooked  bacon
12 leaves butterhead or other crispy lettuce
Kosher  salt and black pepper
1  large onion
6  garlic cloves
1  pickled jalapeño pepper
2 teaspoons Chipotle chile powder
1 tablespoon tomato paste
2 tablespoon Dijon mustard
3/4  cup distilled white vinegar
1 teaspoon paprika
1/3  cup ketchup
2 teaspoons Worcestershire sauce
1/4  cup light brown sugar
Salt  
1  bay leaf
3 pounds pork butt shoulder roast
Hamburger  buns
12 strips bacon
1/2  teaspoon ground black pepper optional
2 tablespoons extra virgin olive oil
1/2  pound Italian sausage removed from casing
1  large zucchini
1 cup chopped onion
3 cloves garlic
2 slices bread pulsed in food processor or blender to make fresh breadcrumbs
2  medium tomatoes
1/4  cup chopped fresh basil
1/2  teaspoon dried oregano
3/4  cup shredded Parmesan cheese
1  large egg
3/4  teaspoon salt
1 teaspoon ground bla

1/3  cup extra virgin olive oil
2 1/2  cups 24
tomato  sauce
xa0make  in step
Tomato  Sauce
2 tablespoons extra virgin olive oil
3/4  cup finely chopped onions
3/4  cup finely chopped carrots
3/4  cup finely chopped celery
2  Tbsp chopped fresh parsley
1 clove garlic
1  28 oz can crushed or whole tomatoes
1/2  teaspoon dried basil
1 teaspoon tomato paste
Salt  and freshly ground black pepper
1 tablespoon extra virgin olive oil
3 pounds pork shoulder
Salt  
1  large onion
1  large carrot
All  of the cloves from 1 head garlic
1 pound hot Italian sausage
1 15 ounce can crushed tomatoes
3 1/2  cups water
2 tablespoons sweet paprika
1 teaspoon hot paprika
1 2 teaspoons smoked paprika
2 15 ounce cans chickpeas
1 bunch parsley
1  1/2  cup 180 g of all purpose flour
1 scant teaspoon Kosher salt
Pinch  of freshly ground black pepper
3  eggs
1  1/2  cup
2 Tbsp melted butter
1 Tbsp vegetable oil
1  lb
4 ounces 
guajillo  
ancho  or a combination of both chili pods
Salt  
1  large 108 ounce 6 lb 1

1 tablespoon freshly grated ginger
Cooked  rice
2 to 2 1/2  pounds boneless skinless chicken thighs
1  large yellow onion
2 stalks celery
2 15 ounce cans tomato puree
1 cup chicken broth
4 cloves garlic
2 tablespoons chili powder
1 teaspoon cumin
1 teaspoon oregano
1 teaspoon salt
2 15 ounce cans black beans
1 cup frozen corn kernels
1 tablespoon cider vinegar
Optional  chili toppings
Diced  green onions
Shredded  cheese
Diced  avocado
Chopped  cilantro
Sour  cream
6 quart slow cooker
1 1/2  pounds chicken breast
1 inch piece fresh ginger
2 cloves garlic
1 tablespoon sesame oil
1 tablespoon soy sauce
1 tablespoon rice wine vinegar
1/4  cup soy sauce
1/4  cup rice wine vinegar
1/3  cup honey
1 tablespoon cornstarch
1/3  cup chicken stock
1 tablespoon sesame seeds
2 teaspoons sesame oil
1 teaspoon chili garlic sauce
Sesame  seeds
To  serve
Cooked  white rice
Scallions  green parts minced for garnish
Sous  vide immersion circulator
Gallon  sized zip top freezer bags
Whole  fryer chicken
S

1  large onion
2  ribs celery
2  carrots
2  bay leaves
4 teaspoon yellow curry powder
1  1/4  pound 570 g
2 cups 475 ml
chicken  stock
2 cups 475 ml water
1 1/2  teaspoons kosher salt
1/4  cup uncooked basmati rice
2  tart apples
1/4   60 ml cup heavy whipping cream
1/4  cup 60 ml plain yogurt
1 tablespoon minced chives
2 pounds 900 g bone
1/2  teaspoon kosher salt
2 cups of cherry tomatoes
6  whole cloves garlic
1 1/2  tablespoons balsamic vinegar
1 1/2  teaspoons extra virgin olive oil
2 sprigs fresh rosemary
2  4 boneless skinless chicken breast halves
1/2  cup flour
1/4  teaspoon salt
Pinch  ground black pepper
3 tablespoons grated Parmesan cheese
4 tablespoons extra virgin olive oil
4 tablespoons butter
1/2  cup chicken stock
2 tablespoons lemon juice
1/4  cup brined capers
2 tablespoons fresh chopped parsley
Chicken  
2  boneless skinless chicken breasts
1/2  teaspoon chili powder
1/2  teaspoon ground cumin
1/4  teaspoon salt
1/4  teaspoon ground black pepper
1 tablespoon extra v

1 3 pound cups cherry or grape tomatoes
8 ounces mozzarella
10  large fresh basil leaves
12  large eggs
1/4  cup chopped green onion
1/2  cup chopped celery
1/2  cup chopped red bell pepper
2 tablespoons Dijon mustard
1/3  cup mayonnaise
1 tablespoon cider white wine or sherry vinegar
1/4  teaspoon Tabasco
1/2  teaspoon paprika
1/2  teaspoon black pepper
1/4  teaspoon salt
xa0  
4 strips bacon
1/2   small white onion
3 cloves garlic
1/2  teaspoon red pepper flakes
1/4  teaspoon salt
1/4  teaspoon black pepper
1/2  cup white wine
3 to 4 cups marinara sauce
1 1/2  to 2 pounds mussels
1 pound spaghetti drained if serving
Crusty  bread
Fresh  parsley
3 cups corn kernels
1 tablespoon butter
1 teaspoon kosher salt
1/2  teaspoon freshly cracked black pepper
2 tablespoons olive oil
2 cups zucchini
3  scallions
4 ounces goat cheese
1/2  cup large basil leaves
4 8 inch flour tortillas or 6 to 8 6 inch corn tortillas
1/2  cup tahini
Joyva  
1/4  cup Greek yogurt
1/2   lemon
2 tablespoons olive oi

...

1 cup 2 sticks unsalted butter
2 pounds 32 ounces powdered sugar
1 teaspoon vanilla extract
1 tablespoon Cointreau
1 cup roughly chopped pecans
Edible  flowers
3  
9 inch cake pans
1 cup 240 ml
10 Tbsp 1 1/4  sticks
1  1/4  cup 250 g sugar
3/4  cup plus 2
1/4  teaspoon salt
1/4  teaspoon cinnamon
2 teaspoons vanilla
2  large eggs
3/4  cup 96 g flour
2/3  cup 90 g chopped pecans
optional  
Vegetable  oil spray
1 medium stalk broccoli stem and crown
5 ounces ham steak
1 1/2  cups grated cheddar
2  large eggs
3 large egg
1 1/2  cups heavy cream
1/4  teaspoon salt
1/8  teaspoon black pepper
9 inch round baking dish or pie pan
2 tablespoons flaxseed meal
1/2  cup plus 2 tablespoons almond milk
2  to 3 very ripe bananas
1/3  cup walnut coconut or avocado oil
3/4  cup sugar
1 teaspoon vanilla extract
1 1/2  cups all purpose flour
1 teaspoon baking soda
Pinch  of salt
4x8 inch loaf pan
2 cups all purpose flour
1 3/4  cups granulated sugar
3/4  cup dark cocoa powder
2 teaspoons baking soda
1 te

1/2  teaspoon nutmeg
1/2  teaspoon allspice
1 cup 240 ml pumpkin purée
1 cup 200 g sugar
4 ounces 1 stick
2  eggs
1/4  cup water
2 teaspoons molasses
1 teaspoon orange zest
1/2  cup 120 ml
Optional  Orange Glaze
1 cup powdered sugar
2 Tbsp orange juice
1 teaspoon orange zest
1/4  teaspoon vanilla
t  use for future use Or if you are working with pumpkin pieces roast or boil them until tender then remove and discard the skin
Blackberries  
2  to 2 1/2  pounds of blackberries
1/4  cup to 1/2  cup of sugar
1 teaspoon lemon juice
1/2  teaspoon lemon zest
Biscuits  
3 cups flour
3 Tbsp sugar
4 teaspoons baking powder
3/4  teaspoon salt
8  Tbsp cold butter
1 1/2  cups heavy cream
1 teaspoon vanilla extract
Whipped  Cream
1 cup heavy whipping cream
2 teaspoons powdered sugar
A  few drops of vanilla extract
1/2  cup butter
1 cup tightly packed light brown sugar
1  egg
1 tablespoon of almond extract
1 cup 128 g all purpose flour
1/2  teaspoon baking powder
1/8  teaspoon baking soda
Pinch  of sal

1  15 5 ounce can coconut milk
1/2  cup water
3 cloves garlic
1  diced serrano
1 teaspoon salt
6 cups spinach
Cooked  rice
Chopped  cilantro
Lime  or lemon wedges to serve
6 quart or larger
4 slices thick sliced bacon
1  medium onion
1 pound green beans
1  14 5 ounce can diced tomatoes
1/3  cup water
1/4  teaspoon salt
1/4  teaspoon ground black pepper
1/8  teaspoon cayenne pepper
2 sprigs fresh thyme
6 quart Instant
or  other electric pressure cooker
3/4  cup pecans
1 batch cornbread
4 tablespoons 1/2  stick unsalted butter
1  large yellow onion
4 stalks celery
2 tablespoons chopped fresh sage
2 tablespoons chopped parsley
1 teaspoon dried thyme
1/2  teaspoon salt
1/4  teaspoon ground black pepper
3/4  cup California
3/4  cup dried cranberries
2  large eggs
1 cup chicken or vegetable stock
1/2  cup half and half
9x13 inch baking dish
1  small butternut squash
2 tablespoons yellow or white miso
1 tablespoon maple syrup
3 tablespoons olive oil
1/4  teaspoon salt
1/4  teaspoon pepper
1 1

2 tablespoons crushed raw pecans
1 tablespoon unsweetened dried shredded coconut
Special  equipment
Blender  
4  small winter squash about 1 1/2  pounds each like kabocha
1/2  tablespoon olive oil
1/4  teaspoon salt
1/4  teaspoon pepper
4 tablespoons olive oil
2 cups brown rice
4 cups low sodium vegetable broth
1 cup diced carrot
1 cup diced shallots
1 cup diced celery
1 1/4  pounds mushrooms
3 cloves garlic
1/2  cup white wine
Pinch  cinnamon
1 cup chopped mixed herbs
2 pounds fingerling potatoes
1/4  cup lemon juice
1/3  cup extra virgin olive oil
1 teaspoon minced fresh thyme leaves
1 teaspoon minced fresh oregano leaves
1/2  teaspoon of Dijon mustard
1 teaspoon salt
1/4  teaspoon freshly ground black pepper
1/2   red onion
1/4  cup dry vermouth
1/4  cup chopped fresh parsley
2 pounds Brussels sprouts
2 tablespoons extra virgin olive oil
Several  large shallots
4 cloves garlic
2 tablespoons plus 2 teaspoons
balsamic  vinegar
Salt  
Pepper  
3/4  teaspoon fresh thyme leaves
1/2  cup 

1  Tbsp minced fresh chives
1/4  teaspoon dried tarragon
1 cup shredded Gruyere
9 large eggs
2  Tbsp milk
1/3  cup grated Parmesan cheese
1/4  teaspoon salt
1/8  teaspoon freshly ground pepper
2  Tbsp extra virgin olive oil
1  medium onion
1  large clove garlic
Sun  dried tomatoes
8  oz 225 g
2  oz 56 g goat cheese
3 tablespoons extra virgin olive oil
1  large yellow onion
1  1/4  pound 600g plum tomatoes
or  
one 14 ounce can of diced tomatoes
6  eggs
Salt  and freshly ground pepper
6  fresh eggs
2  green onions
1  fresh tomato
One  canned jalapeño pepper
2 Tbsp olive oil
Oregano a sprinkling of fresh if available
Salt  one shake from a salt
Cilantro  leaves
12 ounces raw shrimp
1/2  teaspoon paprika
Salt  
1/8  teaspoon freshly ground black pepper
1  Tbsp butter
1/3  cup minced shallots
1  Tbsp minced garlic
1  1/2  cup risotto rice
1/2  cup dry white wine like a Sauvignon Blanc
8 ounces clam juice
1 14 ounce can artichoke hearts
2 3 cups water
2  Tbsp chopped fresh parsley
2 tablesp